Models can only process numbers, so we need to find a way to convert the raw text to numbers. That’s what the tokenizers do, and there are a lot of ways to go about this. The goal is to find the most meaningful representation — that is, the one that makes the most sense to the model — and, if possible, the smallest representation. There are 3 main ways to do this:

- Word based
- Character based
- Subword based

## Word Based Tokenization

The first type of tokenizer that comes to mind is word-based. It’s generally very easy to set up and use with only a few rules, and it often yields decent results. For example, in the image below, the goal is to split the raw text into words and find a numerical representation for each of them:

![Alt Text](images/word_based_tokenization.svg "word_based_tokenization")

There are different ways to split the text. For example, we could use whitespace to tokenize the text into words by applying Python’s split() function:

In [1]:
tokenized_text = "Jim Henson was a puppeteer".split()
print(tokenized_text)

['Jim', 'Henson', 'was', 'a', 'puppeteer']


There are also variations of word tokenizers that have extra rules for punctuation. With this kind of tokenizer, we can end up with some pretty large “vocabularies,” where a vocabulary is defined by the total number of independent tokens that we have in our corpus.

Each word gets assigned an ID, starting from 0 and going up to the size of the vocabulary. The model uses these IDs to identify each word.

### Limitations of Word based Tokenization

If we want to completely cover a language with a word-based tokenizer, we’ll need to have an identifier for each word in the language, which will generate a huge amount of tokens. For example, there are over 500,000 words in the English language, so to build a map from each word to an input ID we’d need to keep track of that many IDs. Furthermore, words like “dog” are represented differently from words like “dogs”, and the model will initially have no way of knowing that “dog” and “dogs” are similar: it will identify the two words as unrelated. The same applies to other similar words, like “run” and “running”, which the model will not see as being similar initially.

Finally, we need a custom token to represent words that are not in our vocabulary. This is known as the “unknown” token, often represented as ”[UNK]” or ”<unk>”. It’s generally a bad sign if you see that the tokenizer is producing a lot of these tokens, as it wasn’t able to retrieve a sensible representation of a word and you’re losing information along the way. The goal when crafting the vocabulary is to do it in such a way that the tokenizer tokenizes as few words as possible into the unknown token.

One way to reduce the amount of unknown tokens is to go one level deeper, using a character-based tokenizer.

## Character-based

Character-based tokenizers split the text into characters, rather than words. This has two primary benefits:

- The vocabulary is much smaller.
- There are much fewer out-of-vocabulary (unknown) tokens, since every word can be built from characters.
But here too some questions arise concerning spaces and punctuation:

### Limitations of Character Based

This approach isn’t perfect either. Since the representation is now based on characters rather than words, one could argue that, intuitively, it’s less meaningful: each character doesn’t mean a lot on its own, whereas that is the case with words. However, this again differs according to the language; in Chinese, for example, each character carries more information than a character in a Latin language.

Another thing to consider is that we’ll end up with a very large amount of tokens to be processed by our model: whereas a word would only be a single token with a word-based tokenizer, it can easily turn into 10 or more tokens when converted into characters.

To get the best of both worlds, we can use a third technique that combines the two approaches: subword tokenization.

## Subword tokenization

> Subword tokenization algorithms rely on the principle that frequently used words should not be split into smaller subwords, but rare words should be decomposed into meaningful subwords.

For instance, “annoyingly” might be considered a rare word and could be decomposed into “annoying” and “ly”. These are both likely to appear more frequently as standalone subwords, while at the same time the meaning of “annoyingly” is kept by the composite meaning of “annoying” and “ly”.

Here is an example showing how a subword tokenization algorithm would tokenize the sequence “Let’s do tokenization!“:

![Alt Text](images/bpe_subword.svg "SubWords")

These subwords end up providing a lot of semantic meaning: for instance, in the example above “tokenization” was split into “token” and “ization”, two tokens that have a semantic meaning while being space-efficient (only two tokens are needed to represent a long word). This allows us to have relatively good coverage with small vocabularies, and close to no unknown tokens.

This approach is especially useful in agglutinative languages such as Turkish, where you can form (almost) arbitrarily long complex words by stringing together subwords.

Unsurprisingly, there are many more techniques out there. To name a few:

- Byte-level BPE, as used in GPT-2
- WordPiece, as used in BERT
- SentencePiece or Unigram, as used in several multilingual models

### 1. Byte-level BPE (Used in GPT-2, GPT-3)

#### What is it?
	•	A variant of Byte Pair Encoding (BPE), applied at the byte level, not character or word level.
	•	First converts input text to UTF-8 bytes, then merges frequent byte pairs into subwords.

#### Key Features:
	•	Handles any text (including Unicode, emojis, code, etc.) without preprocessing.
	•	Operates at the raw byte level, so it doesn’t require knowing any language-specific rules.
	•	GPT-2’s tokenizer (gpt2) is based on this.

#### Example:

Input: "hello 🙂"

→ Bytes: [104, 101, 108, 108, 111, 32, 240, 159, 153, 130]

→ Tokens: ["hello", "Ġ🙂"] (GPT-style tokenization)

### 2. WordPiece (Used in BERT, DistilBERT, RoBERTa)

#### What is it?
	•	A subword tokenization algorithm originally developed for Google’s BERT models.
	•	Trains by greedily combining frequent character-based subwords to reduce vocabulary size.

#### Key Features:
	•	Produces tokens that are prefix-split, like "##ing" or "##ly" — the ## indicates a subword continuation.
	•	Requires whitespace-tokenized input (i.e., assumes some knowledge of word boundaries).
	•	Slightly more linguistically aware than byte-level BPE.

#### Example:

Input: "unbelievable"

→ Tokens: ["un", "##believable"]


### 3. Unigram Language Model (used in SentencePiece)

(Used in T5, mT5, ALBERT, XLM-R)

#### What is it?
	•	A probabilistic tokenization algorithm used by SentencePiece.
	•	Trains a language model on possible subword sequences and selects the most likely sequence of tokens for a given sentence.

#### Key Features:
	•	Doesn’t assume pre-tokenization or whitespace.
	•	Often used in multilingual models where consistency across scripts (Latin, Cyrillic, Chinese) is important.
	•	More flexible than BPE — can drop tokens from the vocabulary if it improves overall likelihood.

#### Example:

Input: "unbelievable"

→ Could be tokenized as: ["un", "believable"] or ["unbeliev", "able"], depending on model’s learned probabilities

## Loading and saving

Loading and saving tokenizers is as simple as it is with models. Actually, it’s based on the same two methods: from_pretrained() and save_pretrained(). These methods will load or save the algorithm used by the tokenizer (a bit like the architecture of the model) as well as its vocabulary (a bit like the weights of the model).

Loading the BERT tokenizer trained with the same checkpoint as BERT is done the same way as loading the model, except we use the BertTokenizer class:

In [3]:
from transformers import BertTokenizer

tokenizer = BertTokenizer.from_pretrained("bert-base-cased")

Similar to AutoModel, the AutoTokenizer class will grab the proper tokenizer class in the library based on the checkpoint name, and can be used directly with any checkpoint:

In [4]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("bert-base-cased")

In [5]:
tokenizer("Using a Transformer network is simple")

{'input_ids': [101, 7993, 170, 13809, 23763, 2443, 1110, 3014, 102], 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0, 0], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1]}

## Encoding

Translating text to numbers is known as encoding. Encoding is done in a two-step process: 
- the tokenization, 
- followed by the conversion to input IDs.

As we’ve seen, the first step is to split the text into words (or parts of words, punctuation symbols, etc.), usually called tokens. There are multiple rules that can govern that process, which is why we need to instantiate the tokenizer using the name of the model, to make sure we use the same rules that were used when the model was pretrained.

The second step is to convert those tokens into numbers, so we can build a tensor out of them and feed them to the model. To do this, the tokenizer has a vocabulary, which is the part we download when we instantiate it with the from_pretrained() method. Again, we need to use the same vocabulary used when the model was pretrained.

To get a better understanding of the two steps, we’ll explore them separately. Note that we will use some methods that perform parts of the tokenization pipeline separately to show you the intermediate results of those steps, but in practice, you should call the tokenizer directly on your inputs (as shown in the section 2).

### Tokenization

The tokenization process is done by the tokenize() method of the tokenizer:

In [12]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("bert-base-cased")

sequence = ["I've been waiting for a Hugging Face course my whole life.", "I hate this so much!"]

tokens = tokenizer.tokenize(sequence)
print(tokens)

['I', "'", 've', 'been', 'waiting', 'for', 'a', 'Hu', '##gging', 'Face', 'course', 'my', 'whole', 'life', '.', 'I', 'hate', 'this', 'so', 'much', '!']


### From tokens to input IDs

The conversion to input IDs is handled by the convert_tokens_to_ids() tokenizer method:

In [13]:
ids = tokenizer.convert_tokens_to_ids(tokens)
print(ids)


[146, 112, 1396, 1151, 2613, 1111, 170, 20164, 10932, 10289, 1736, 1139, 2006, 1297, 119, 146, 4819, 1142, 1177, 1277, 106]


### Decoding

Decoding is going the other way around: from vocabulary indices, we want to get a string. This can be done with the decode() method as follows:

In [17]:
decoded_string = tokenizer.decode(ids)
print(decoded_string)

I ' ve been waiting for a Hugging Face course my whole life. I hate this so much!


Note that the decode method not only converts the indices back to tokens, but also groups together the tokens that were part of the same words to produce a readable sentence. This behavior will be extremely useful when we use models that predict new text (either text generated from a prompt, or for sequence-to-sequence problems like translation or summarization).